<a href="https://colab.research.google.com/github/chielgroen1998/MLOps/blob/main/datacollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install alpha_vantage
#!pip install xgboost
#!pip install pandas_ta==0.3.14b0
#!pip install optuna
# First install compatible versions
#!pip install --upgrade pyspark==3.3.0 delta-spark==2.2.0
#!pip install --upgrade numpy
#!pip install --upgrade pandas_ta


In [2]:
import pandas as pd
import numpy as np
# Fix for pandas_ta import issue
np.NaN = np.nan  # Create NaN as an alias to nan
import pandas_ta as ta
import optuna
import logging
from datetime import timedelta
from alpha_vantage.timeseries import TimeSeries

add more data visualizations and insights

after engineering move to pyspark, describe in report why

transforming (remove nulls after cuz its about loading the data in first)

removing rows or mean imputation without a rolling window is not smart because of the timeseries nature of stocks, it would ruin indications of growing or shrinking trends in price


In [3]:
# Step 3: Define your API key and list of symbols
api_key = 'PH8UF085L5T5MFZN'
symbols = ['LIT', 'REMX', 'URA', 'GDX', 'GDXJ', 'XLE', 'ICLN', 'SPY', 'GLD', 'CPER', 'USO', 'UNG', 'KOL', 'JJN']

# Step 4: Fetch the data for each symbol using Alpha Vantage API
ts = TimeSeries(key=api_key, output_format='pandas')
all_data = {}
for symbol in symbols:
    data, meta_data = ts.get_daily(symbol=symbol, outputsize='full')
    # Filter data from 2018 onwards
    data.index = pd.to_datetime(data.index)
    data = data[data.index >= '2023-01-01']
    all_data[symbol] = data
    print(f"Data for {symbol}:")
    print(data.head())  # Print the first few rows of the dataset
    # Plot the data
    data['4. close'].plot(figsize=(12, 6), title=f'{symbol} Daily Closing Price (from 2018)')
    plt.xlabel('Date')
    plt.ylabel('Closing Price')
    plt.grid(True)
    plt.show()

ValueError: We have detected your API key as PH8UF085L5T5MFZN and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.

In [ ]:
# Rename columns to match pandas_ta requirements
all_data['LIT'].columns = ['open', 'high', 'low', 'close', 'volume']
all_data['REMX'].columns = ['open', 'high', 'low', 'close', 'volume']
all_data['URA'].columns = ['open', 'high', 'low', 'close', 'volume']
all_data['GDX'].columns = ['open', 'high', 'low', 'close', 'volume']
all_data['GDXJ'].columns = ['open', 'high', 'low', 'close', 'volume']
all_data['XLE'].columns = ['open', 'high', 'low', 'close', 'volume']
all_data['ICLN'].columns = ['open', 'high', 'low', 'close', 'volume']
all_data['SPY'].columns = ['open', 'high', 'low', 'close', 'volume']
all_data['GLD'].columns = ['open', 'high', 'low', 'close', 'volume']
all_data['CPER'].columns = ['open', 'high', 'low', 'close', 'volume']
all_data['USO'].columns = ['open', 'high', 'low', 'close', 'volume']
all_data['UNG'].columns = ['open', 'high', 'low', 'close', 'volume']
all_data['KOL'].columns = ['open', 'high', 'low', 'close', 'volume']
all_data['JJN'].columns = ['open', 'high', 'low', 'close', 'volume']

In [25]:
# Apply technical indicators to each dataframe
for symbol, data in all_data.items():
    # Trend Indicators
    data.ta.sma(length=100, append=True)               # Simple Moving Average
    data.ta.ema(length=20, append=True)               # Exponential Moving Average
    data.ta.wma(length=20, append=True)               # Weighted Moving Average
    data.ta.hma(length=20, append=True)               # Hull Moving Average
    data.ta.vwma(length=20, append=True)              # Volume-Weighted Moving Average
    data.ta.macd(fast=12, slow=26, signal=9, append=True)  # MACD
    data.ta.adx(length=14, append=True)               # Average Directional Index
    data.ta.cci(length=20, append=True)               # Commodity Channel Index
    data.ta.rsi(length=14, append=True)               # Relative Strength Index
    data.ta.stoch(k=14, d=3, append=True)             # Stochastic Oscillator
    data.ta.ichimoku(tenkan=9, kijun=26, senkou=52, append=True)  # Ichimoku Cloud
    data.ta.supertrend(length=7, multiplier=3, append=True)  # SuperTrend
    data.ta.psar(step=0.02, max_step=0.2, append=True)  # Parabolic SAR

    # Momentum Indicators
    data.ta.mom(length=50, append=True)               # Momentum
    data.ta.roc(length=10, append=True)               # Rate of Change
    data.ta.willr(length=14, append=True)             # Williams %R
    data.ta.uo(append=True)                           # Ultimate Oscillator
    data.ta.ao(fast=5, slow=34, append=True)          # Awesome Oscillator
    data.ta.kama(length=10, append=True)              # Kaufman's Adaptive MA
    data.ta.cg(length=10, append=True)                # Center of Gravity

    # Volatility Indicators
    data.ta.bbands(length=20, std=2, append=True)     # Bollinger Bands
    data.ta.atr(length=14, append=True)               # Average True Range
    data.ta.kc(length=20, scalar=2, append=True)      # Keltner Channel
    data.ta.donchian(lower_length=20, upper_length=20, append=True)  # Donchian Channel
    data.ta.rvi(length=14, append=True)               # Relative Volatility Index

    # Volume Indicators
    data.ta.obv(append=True)                          # On-Balance Volume
    data.ta.cmf(length=20, append=True)               # Chaikin Money Flow
    data.ta.adosc(fast=3, slow=10, append=True)       # AD Oscillator
    data.ta.mfi(length=14, append=True)               # Money Flow Index
    data.ta.vwap(append=True)                         # VWAP
    data.ta.eom(length=14, append=True)               # Ease of Movement
    data.ta.nvi(length=255, append=True)              # Negative Volume Index

    # Other Indicators
    data.ta.squeeze(append=True)                      # TTM Squeeze
    data.ta.fisher(length=9, append=True)             # Fisher Transform
    data.ta.decay(length=5, mode="linear", append=True)  # Linear Decay
    data.ta.decay(length=5, mode="exponential", append=True)  # Exponential Decay
    data.ta.vortex(length=14, append=True)            # Vortex Indicator
    data.ta.zscore(length=20, append=True)            # Z-Score
    data.ta.entropy(length=10, append=True)           # Entropy

    print(f"\nData for {symbol}:")
    print(data.head())  # Print the first few rows of the dataset
    # Plot the data
    data['close'].plot(figsize=(12, 6), title=f'{symbol} Daily Closing Price (from 2018)')
    plt.xlabel('Date')
    plt.ylabel('Closing Price')
    plt.grid(True)
    plt.show()
    
    # Print the number of technical indicators added
    print(f"Number of technical indicators added to {symbol}: {len(data.columns) - 5}")  # Subtract 5 for OHLCV columns

ValueError: Length mismatch: Expected axis has 71 elements, new values have 5 elements

,open,high,low,close,volume
date,,,,,
2025-05-16,39.41,39.4100,38.9400,39.31,103116.0
2025-05-15,39.65,39.8600,39.1900,39.67,141722.0
2025-05-14,40.08,40.2900,39.6337,39.77,126862.0
2025-05-13,39.86,40.1000,39.6200,39.93,52374.0
2025-05-12,40.10,40.3089,39.5550,39.67,123388.0
...,...,...,...,...,...
2018-01-08,32.00,32.0300,31.7900,31.93,321224.0
2018-01-05,31.54,31.8600,31.5400,31.86,306516.0
2018-01-04,31.55,31.6500,31.4700,31.53,291952.0


In [ ]:
    # Apply technical indicators to each dataframe
    # Trend Indicators
    data.ta.sma(length=100, append=True)               # Simple Moving Average
    data.ta.ema(length=20, append=True)               # Exponential Moving Average
    data.ta.wma(length=20, append=True)               # Weighted Moving Average
    data.ta.hma(length=20, append=True)               # Hull Moving Average
    data.ta.vwma(length=20, append=True)              # Volume-Weighted Moving Average
    data.ta.macd(fast=12, slow=26, signal=9, append=True)  # MACD
    data.ta.adx(length=14, append=True)               # Average Directional Index
    data.ta.cci(length=20, append=True)               # Commodity Channel Index
    data.ta.rsi(length=14, append=True)               # Relative Strength Index
    data.ta.stoch(k=14, d=3, append=True)             # Stochastic Oscillator
    data.ta.ichimoku(tenkan=9, kijun=26, senkou=52, append=True)  # Ichimoku Cloud
    data.ta.supertrend(length=7, multiplier=3, append=True)  # SuperTrend
    data.ta.psar(step=0.02, max_step=0.2, append=True)  # Parabolic SAR

    # Momentum Indicators
    data.ta.mom(length=50, append=True)               # Momentum
    data.ta.roc(length=10, append=True)               # Rate of Change
    data.ta.willr(length=14, append=True)             # Williams %R
    data.ta.uo(append=True)                           # Ultimate Oscillator
    data.ta.ao(fast=5, slow=34, append=True)          # Awesome Oscillator
    data.ta.kama(length=10, append=True)              # Kaufman's Adaptive MA
    data.ta.cg(length=10, append=True)                # Center of Gravity

    # Volatility Indicators
    data.ta.bbands(length=20, std=2, append=True)     # Bollinger Bands
    data.ta.atr(length=14, append=True)               # Average True Range
    data.ta.kc(length=20, scalar=2, append=True)      # Keltner Channel
    data.ta.donchian(lower_length=20, upper_length=20, append=True)  # Donchian Channel
    data.ta.rvi(length=14, append=True)               # Relative Volatility Index

    # Volume Indicators
    data.ta.obv(append=True)                          # On-Balance Volume
    data.ta.cmf(length=20, append=True)               # Chaikin Money Flow
    data.ta.adosc(fast=3, slow=10, append=True)       # AD Oscillator
    data.ta.mfi(length=14, append=True)               # Money Flow Index
    data.ta.vwap(append=True)                         # VWAP
    data.ta.eom(length=14, append=True)               # Ease of Movement
    data.ta.nvi(length=255, append=True)              # Negative Volume Index

    # Other Indicators
    data.ta.squeeze(append=True)                      # TTM Squeeze
    data.ta.fisher(length=9, append=True)             # Fisher Transform
    data.ta.decay(length=5, mode="linear", append=True)  # Linear Decay
    data.ta.decay(length=5, mode="exponential", append=True)  # Exponential Decay
    data.ta.vortex(length=14, append=True)            # Vortex Indicator
    data.ta.zscore(length=20, append=True)            # Z-Score
    data.ta.entropy(length=10, append=True)           # Entropy

    all_data[symbol] = data
    print(f"\nData for {symbol}:")
    print(data.head())  # Print the first few rows of the dataset
    # Plot the data
    data['close'].plot(figsize=(12, 6), title=f'{symbol} Daily Closing Price (from 2018)')
    plt.xlabel('Date')
    plt.ylabel('Closing Price')
    plt.grid(True)
    plt.show()
    
    # Print the number of technical indicators added
    print(f"Number of technical indicators added to {symbol}: {len(data.columns) - 5}")  # Subtract 5 for OHLCV columns

In [15]:
data

,open,high,low,close,volume
date,,,,,
2023-06-14,28.4,28.4,28.4,28.4,0.0
2023-06-13,28.4,28.4,28.4,28.4,0.0
2023-06-12,28.4,28.4,28.4,28.4,0.0
2023-06-09,28.4,28.4,28.4,28.4,0.0
2023-06-08,28.4,28.4,28.4,28.4,83.0
...,...,...,...,...,...
2018-01-24,50.6,50.6,50.6,50.6,0.0
2018-01-23,50.6,50.6,50.6,50.6,0.0
2018-01-22,50.6,50.6,50.6,50.6,0.0


In [ ]:
#filling on NaN's, but is in SPARK,so want to remove


# def fill_nan_with_rolling_median(df, window_size=21): # Use an odd window size to center
#     if window_size % 2 == 0:
#         window_size += 1
#     if window_size < 3:
#         window_size = 3

#     center_offset = (window_size - 1) // 2
#     df_filled = df.apply(lambda col: col.fillna(col.rolling(window=window_size, center=True).median()))


#     df_filled = df_filled.fillna(df_filled.median())

#     return df_filled

# train_data_filled = fill_nan_with_rolling_median(train_data.copy(), window_size=21)
# test_data_filled = fill_nan_with_rolling_median(test_data.copy(), window_size=21)

# print("NaN counts in train_data_filled after rolling median imputation:")
# print(train_data_filled.isnull().sum())

# print("\nNaN counts in test_data_filled after rolling median imputation:")
# print(test_data_filled.isnull().sum())

# # Now you can save these filled dataframes to CSVs and load them into PySpark
# train_data_filled.to_csv("train_filled.csv", index=True)
# test_data_filled.to_csv("test_filled.csv", index=True)

# # Load filled data from CSV into PySpark DataFrame
# train_spark_df = spark.read.csv("train_filled.csv", header=True, inferSchema=True)
# test_spark_df = spark.read.csv("test_filled.csv", header=True, inferSchema=True)

# # Show schema to verify data types
# train_spark_df.printSchema()
# test_spark_df.printSchema()

# # Show first few rows
# train_spark_df.show(5)
# test_spark_df.show(5)

NaN counts in train_data_filled after rolling median imputation:
open              0
high              0
low               0
close             0
volume            0
                 ..
VTXM_14           0
ZS_20             0
ENTP_10           0
conditions_met    0
label             0
Length: 73, dtype: int64

NaN counts in test_data_filled after rolling median imputation:
open              0
high              0
low               0
close             0
volume            0
                 ..
VTXM_14           0
ZS_20             0
ENTP_10           0
conditions_met    0
label             0
Length: 73, dtype: int64
root
 |-- date: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- SMA_100: double (nullable = true)
 |-- EMA_20: double (nullable = true)
 |-- WMA_20: double (nullable = true)
 |-- HMA_20: double (nullable = true)
 

move from SKLEARN to pysparkML